In [167]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import ssl
import pandas as pd
import urllib.request
from PIL import Image
import time
import shutil
import requests



In [168]:
ssl._create_default_https_context = ssl._create_unverified_context

In [176]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT DISTINCT ?country  ?capital ?country_name ?capital_name ?country_flag ?country_abstract
    WHERE {
      ?country a yago:WikicatCountries , dbo:Country.
      ?country dbo:capital ?capital.
      ?country rdfs:label ?country_name.
      ?capital rdfs:label ?capital_name.
      ?country dbo:thumbnail ?country_flag.
      ?country dbo:abstract ? ?country_abstract.

      FILTER(lang(?country_name) = 'en')
      FILTER(lang(?capital_name) = 'en')
      FILTER(lang(?country_abstract) = 'en')

    } 
    
    ORDER BY ?country
        
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results["results"].keys())

d = {
     "Country_uri" : [],
     "Country_name" : [],
     "Country_flag" : [], 
     "Capital_uri" : [], 
     "Capital_name" : [], 
     "Country_abstract" : []
    }

for result in results["results"]["bindings"]:
  d["Country_uri"].append(result["country"]["value"])
  d["Country_name"].append(result["country_name"]["value"])
  d["Country_flag"].append(result["country_flag"]["value"])
  d["Capital_uri"].append(result["capital"]["value"])
  d["Capital_name"].append(result["capital_name"]["value"])
  d["Country_abstract"].append(result["country_abstract"]["value"])

df_cities = pd.DataFrame(d)

df_cities[151]["Country_flag"]


dict_keys(['distinct', 'ordered', 'bindings'])


,Country_uri,Country_name,Country_flag,Capital_uri,Capital_name,Country_abstract
0,http://dbpedia.org/resource/Afghanistan,Afghanistan,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Kabul,Kabul,"Afghanistan (/æfˈɡænɪstæn, æfˈɡɑːnɪstɑːn/), of..."
1,http://dbpedia.org/resource/Albania,Albania,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Tirana,Tirana,"Albania (/ælˈbeɪniə, ɔːl-/ a(w)l-BAY-nee-ə; Al..."
2,http://dbpedia.org/resource/Algeria,Algeria,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Algiers,Algiers,"Algeria, officially the People's Democratic Re..."
3,http://dbpedia.org/resource/Andorra,Andorra,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Andorra_la_Vella,Andorra la Vella,"Andorra, officially the Principality of Andorr..."
4,http://dbpedia.org/resource/Angola,Angola,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Luanda,Luanda,"Angola (/ænˈɡoʊlə/; Portuguese: [ɐ̃ˈɡɔlɐ]), of..."
...,...,...,...,...,...,...
150,http://dbpedia.org/resource/South_Sudan,South Sudan,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Juba,Juba,"South Sudan (/suːˈdɑːn, -ˈdæn/), officially kn..."
151,http://dbpedia.org/resource/Spain,Spain,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Madrid,Madrid,"Spain (Spanish: España, [esˈpaɲa]), officially..."
152,http://dbpedia.org/resource/Sri_Lanka,Sri Lanka,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Sri_Jayawardenepur...,Sri Jayawardenepura Kotte,"Sri Lanka (UK: /sri ˈlæŋkə, ʃriː -/, US: /- ˈl..."
153,http://dbpedia.org/resource/Sri_Lanka,Sri Lanka,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Colombo,Colombo,"Sri Lanka (UK: /sri ˈlæŋkə, ʃriː -/, US: /- ˈl..."


In [170]:
#Printing random country with capital and flag

row = df_cities.sample(ignore_index=True).iloc[0]
country_name = row.loc["Country_name"]
country_capital = row["Capital_name"]
flag_link = row["Country_flag"]

print(f"Pays : {country_name}")
print(f"Capitale : {country_capital}")
  
urllib.request.urlretrieve(flag_link, "temp_flag.png")
  
img = Image.open("temp_flag.png")
img.show()


Pays : South Korea
Capitale : Seoul


In [174]:
# Downloading all flags for all countries

done = [] # for coutries that appear twice or more in the dataset (multiple capitals)

for index, row in df_cities.iterrows():
    time.sleep(1)
    country = row["Country_name"]
    print(country)
    if country not in done:
        url = row["Country_flag"].encode("ascii", "ignore").decode("ascii")
        response = requests.get(url, stream=True)
        with open("./country_flags/" + country + "_flag.png", 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        done.append(country)

Afghanistan
Albania
Algeria
Andorra
Angola
Armenia
Australia
Austria
Azerbaijan
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Benin
Bhutan
Bolivia
Bolivia
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Central African Republic
Chile
Colombia
Comoros
Costa Rica
Croatia
Cyprus
Czech Republic
Democratic Republic of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
East Timor
Ecuador
Equatorial Guinea
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Federated States of Micronesia
Fiji
Finland
France
Gabon
Georgia (country)
Germany
Ghana
Greece
Grenada
Guatemala
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Israel
Israel
Italy
Ivory Coast
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Madagascar
Malawi
Malaysia
Malaysia
Maldives
Mali
Marshall Islands
Mauritania
Mauritius
Moldova
Monaco
Monaco
Mongolia